In [1]:
print("hello world")

hello world


In [2]:
import torch
isAvailable = torch.cuda.is_available()
print("Is cuda available ",isAvailable)

Is cuda available  False


In [1]:
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
config = {
        "batch_size": tune.choice([8, 16, 32]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        # red convolucional
        "cant_filtros_conv1": tune.choice([6, 12, 18]),
        "kernel_size_maxpool1": tune.choice([2, 3]),
        "cant_filtros_conv2": tune.choice([16, 22, 28]),
        "kernel_size_maxpool2": tune.choice([2, 3]),
        "full_l1": tune.choice([120, 140, 160]),

        "full_l2": tune.choice([84, 104, 124])
    }

In [4]:
"""
probar que absolutamente todas las posibles combinaciones no tiran fallo.
"""
import torch
from modelo_convolucional import instanciarRed
passed = 0
not_passed =0
fail_configs = []
for conv1 in [6, 12, 18]:
    for maxpool1 in [2, 3]:
        for conv2 in [16, 22, 28]:
            for maxpool2 in [2, 3]:
                for l1 in [120, 140, 160]:
                    for l2 in [84, 104, 124]:
                        config = {
                            "cant_filtros_conv1": conv1,
                            "kernel_size_maxpool1": maxpool1,
                            "cant_filtros_conv2": conv2,
                            "kernel_size_maxpool2": maxpool2,
                            "full_l1": l1,
                            "full_l2": l2,
                        }

                        model = instanciarRed(config)
                        random = torch.rand(1, 3, 32, 32)
                        try:
                            model(random)
                            passed+=1
                        except Exception as e:
                            not_passed+=1
                            print(config)
                            fail_configs.append(config)
                            #raise e
print("passed ",passed)
print("not passed ",not_passed)
print(not_passed/(passed+not_passed)*100,"% not passed")

passed  324
not passed  0
0.0 % not passed


In [2]:
import torch
from modelo_convolucional import instanciarRed
# config = fail_configs[0]
# print(config)
config = {
    "cant_filtros_conv1": 6,
    "kernel_size_maxpool1": 2,
    "cant_filtros_conv2": 16,
    "kernel_size_maxpool2": 2,
    "full_l1": 120,
    "full_l2": 84,
}

model = instanciarRed(config)

random = torch.rand(1, 3, 32, 32)




In [3]:
"""
replicar un proceso forward paso a paso, revisando las dimensiones, en busca del error dimensional.
"""
from icecream import ic
from torch import nn
# conv1
ic("conv1")
ic(model.o_conv2d)
conv = nn.Conv2d(in_channels=3, out_channels=model.o_conv2d[0],
                 kernel_size=5, stride=1, padding=1)
ic(random.shape)
rta = conv(random)
ic(rta.shape)
# maxpool1
ic("maxpool1")
maxPool1 = nn.MaxPool2d(kernel_size=config["kernel_size_maxpool1"], stride=2)
rta = maxPool1(rta)
ic(model.o_maxPool)
ic(rta.shape)
# conv2
ic("conv2")
conv2 = nn.Conv2d(in_channels=model.o_maxPool[0], out_channels=config["cant_filtros_conv2"],
                  kernel_size=5, stride=1, padding=0)
rta = conv2(rta)
ic(model.o_conv2d2)
ic(rta.shape)

ic(model.layer2)


ic| 'conv1'
ic| model.o_conv2d: [6, 30]
ic| random.shape: torch.Size([1, 3, 32, 32])
ic| rta.shape: torch.Size([1, 6, 30, 30])
ic| 'maxpool1'
ic| model.o_maxPool: [6, 15]
ic| rta.shape: torch.Size([1, 6, 15, 15])
ic| 'conv2'
ic| model.o_conv2d2: [16, 11]
ic| rta.shape: torch.Size([1, 16, 11, 11])
ic| model.layer2: Sequential(
                    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
                    (1): ReLU()
                    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
                  )


Sequential(
  (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [2]:
# ver dimensiones de modelos finales

from torchinfo import summary
from modelo_convolucional import instanciarModeloConvolucional
from modelo_mlp import instanciarModeloMLP
config = {
    "batch_size": 16,
    "learning_rate": 0.000106647,
    "cant_filtros_conv1": 18,
    "kernel_size_maxpool1": 2,
    "cant_filtros_conv2": 28,
    "kernel_size_maxpool2": 3,
    "full_l1": 160,
    "full_l2": 104,
    "weight_decay":1e-4,
}

model = instanciarModeloConvolucional(config)
dimension_imagen = [3,32,32]
resumen = summary(model, input_size=(config["batch_size"], *dimension_imagen))
print("Modelo convolucional")
print(resumen)

model = instanciarModeloMLP(config)
resumen = summary(model, input_size=(config["batch_size"], *dimension_imagen))
print("Modelo MLP")
print(resumen)


Modelo convolucional
Layer (type:depth-idx)                   Output Shape              Param #
RedConvolucional                         [16, 10]                  --
├─Sequential: 1-1                        [16, 18, 15, 15]          --
│    └─Conv2d: 2-1                       [16, 18, 30, 30]          1,368
│    └─ReLU: 2-2                         [16, 18, 30, 30]          --
│    └─MaxPool2d: 2-3                    [16, 18, 15, 15]          --
├─Sequential: 1-2                        [16, 28, 5, 5]            --
│    └─Conv2d: 2-4                       [16, 28, 11, 11]          12,628
│    └─ReLU: 2-5                         [16, 28, 11, 11]          --
│    └─MaxPool2d: 2-6                    [16, 28, 5, 5]            --
├─Sequential: 1-3                        [16, 10]                  --
│    └─Flatten: 2-7                      [16, 700]                 --
│    └─Linear: 2-8                       [16, 160]                 112,160
│    └─ReLU: 2-9                         [16, 160]  